In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
from transformers import AutoModelForCausalLM
import json
import torch




MODEL_NAME = "qwen-3b-raw"



BATCH_SIZE = 4

SYSTEM = {"role": "system", "content": "Ты — экспертная система Compressa RAG. Предоставляющая точные и релевантные ответы на вопросы."}

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct", padding_side='left')

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct",
    device_map="auto",)
model.eval()


REJECT_ANSW = "К сожалению, ответа на вопрос нет в упомянутых источниках"

def get_summary_prompt(context_list, question):
    context = ''
    for i, c in enumerate(context_list):
        context += f'Источник [{i+1}], документ {c["doc"]}:'+"\n"+c['text']+"\n\n"

    prompt = (
        f"# Контекстная информация:\n\n{context}\n\n"
        "---\n"
        "# Инструкции:\n\n"
        "1. Дай краткий ответ на вопрос, используя только информацию из контекста.\n"
        f"2. Если ответа на вопрос нет в источниках, напиши: \"{REJECT_ANSW}\".\n"
        f"# Вопрос:\n\n{question}\n\n"
    )

    return prompt

def generate_conversation(row):
    formatted_message = [SYSTEM] + [
        {"role": "user", "content": get_summary_prompt(row['context'], row['table_data']['Вопрос'])},
    ]
    return formatted_message

2025-06-15 15:01:19.725731: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749988879.744190    4244 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749988879.750185    4244 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749988879.766425    4244 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749988879.766440    4244 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749988879.766443    4244 computation_placer.cc:177] computation placer alr

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
device = model.device


for folder_name in ["", "_random", "_reverse"]:
    for type in ["large", "medium", "small"]:

        with open(f"generation{folder_name}/benchmark/benchmark_short_{type}.json", "r") as f:
            benchmark_data = json.load(f)

        if type == "small":
            max_seq_length = 2800
        elif type == "medium":
            max_seq_length = 4900
        else:
            max_seq_length = 9100

        print(type)

        for i in tqdm(range(0, len(benchmark_data), BATCH_SIZE)):
            batch = benchmark_data[i:i+BATCH_SIZE]
            prompts = [generate_conversation(row) for row in batch]
            chat_prompts = tokenizer.apply_chat_template(
                prompts,
                tokenize=False,
                add_generation_prompt=True
            )
                
            inputs = tokenizer(
                chat_prompts,
                padding=True,
                truncation=True,
                return_tensors="pt",
                max_length=max_seq_length
            ).to(device)
            
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=1024,
                    do_sample=False,
                    temperature=None,
                    top_p=None,
                    top_k=None
                )
            
            decoded = tokenizer.batch_decode(
                outputs[:, inputs.input_ids.shape[1]:], 
                skip_special_tokens=True,
            )
            
            for j, item in enumerate(batch):
                # print(batch[j]['table_data']['Вопрос'])
                # print(f"---------------------------------------------------\n{decoded[j]}\n---------------------------------------------------")
                # print(batch[j]['table_data']['Ответ'])
                # print()
                # print()
                # print()

                item["model_answer"] = decoded[j]

            # assert(0 == 1)


        with open(f"generation{folder_name}/{MODEL_NAME}_{type}.json", "w") as f:
            json.dump(benchmark_data, f, ensure_ascii=False, indent=2)

        torch.cuda.empty_cache()

large


100%|██████████| 58/58 [1:08:45<00:00, 71.13s/it] 


medium


100%|██████████| 58/58 [32:40<00:00, 33.80s/it]


small


...

large


100%|██████████| 58/58 [1:06:50<00:00, 69.15s/it] 


medium


 40%|███▉      | 23/58 [15:18<23:18, 39.95s/it]


KeyboardInterrupt: 

In [2]:
device = model.device


for folder_name in ["_reverse"]:
    for type in ["medium", "small"]:

        if folder_name == "_random" and type == "large":
            continue

        with open(f"generation{folder_name}/benchmark/benchmark_short_{type}.json", "r") as f:
            benchmark_data = json.load(f)

        if type == "small":
            max_seq_length = 2800
        elif type == "medium":
            max_seq_length = 4900
        else:
            max_seq_length = 9100

        print(type)

        for i in tqdm(range(0, len(benchmark_data), BATCH_SIZE)):
            batch = benchmark_data[i:i+BATCH_SIZE]
            prompts = [generate_conversation(row) for row in batch]
            chat_prompts = tokenizer.apply_chat_template(
                prompts,
                tokenize=False,
                add_generation_prompt=True
            )
                
            inputs = tokenizer(
                chat_prompts,
                padding=True,
                truncation=True,
                return_tensors="pt",
                max_length=max_seq_length
            ).to(device)
            
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=1024,
                    do_sample=False,
                    temperature=None,
                    top_p=None,
                    top_k=None
                )
            
            decoded = tokenizer.batch_decode(
                outputs[:, inputs.input_ids.shape[1]:], 
                skip_special_tokens=True,
            )
            
            for j, item in enumerate(batch):
                # print(batch[j]['table_data']['Вопрос'])
                # print(f"---------------------------------------------------\n{decoded[j]}\n---------------------------------------------------")
                # print(batch[j]['table_data']['Ответ'])
                # print()
                # print()
                # print()

                item["model_answer"] = decoded[j]

            # assert(0 == 1)


        with open(f"generation{folder_name}/{MODEL_NAME}_{type}.json", "w") as f:
            json.dump(benchmark_data, f, ensure_ascii=False, indent=2)

        torch.cuda.empty_cache()

medium


  0%|          | 0/58 [00:00<?, ?it/s]

100%|██████████| 58/58 [35:11<00:00, 36.41s/it]


small


100%|██████████| 58/58 [20:06<00:00, 20.80s/it]


# raft

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
from transformers import AutoModelForCausalLM
import json
import torch




MODEL_NAME = "qwen-3b-raft"



BATCH_SIZE = 4

SYSTEM = {"role": "system", "content": "Ты — экспертная система Compressa RAG. Предоставляющая точные и релевантные ответы на вопросы."}

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct", padding_side='left')

model = AutoModelForCausalLM.from_pretrained(
    "qwen3b-v100-webglm-raft/merged_model",
    device_map="auto",)
model.eval()


REJECT_ANSW = "К сожалению, ответа на вопрос нет в упомянутых источниках"

def get_summary_prompt(context_list, question):
    context = ''
    for i, c in enumerate(context_list):
        context += f'Источник [{i+1}], документ {c["doc"]}:'+"\n"+c['text']+"\n\n"

    prompt = (
        f"# Контекстная информация:\n\n{context}\n\n"
        "---\n"
        "# Инструкции:\n\n"
        "1. Дай краткий ответ на вопрос, используя только информацию из контекста.\n"
        f"2. Если ответа на вопрос нет в источниках, напиши: \"{REJECT_ANSW}\".\n"
        f"# Вопрос:\n\n{question}\n\n"
    )

    return prompt

def generate_conversation(row):
    formatted_message = [SYSTEM] + [
        {"role": "user", "content": get_summary_prompt(row['context'], row['table_data']['Вопрос'])},
    ]
    return formatted_message

2025-06-15 16:10:48.884157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749993048.907386   16844 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749993048.915313   16844 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749993048.934850   16844 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749993048.934870   16844 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749993048.934873   16844 computation_placer.cc:177] computation placer alr

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
device = model.device


for folder_name in ["", "_random", "_reverse"]:
    for type in ["large", "medium", "small"]:

        with open(f"generation{folder_name}/benchmark/benchmark_short_{type}.json", "r") as f:
            benchmark_data = json.load(f)

        if type == "small":
            max_seq_length = 2800
        elif type == "medium":
            max_seq_length = 4900
        else:
            max_seq_length = 9100

        print(type)

        for i in tqdm(range(0, len(benchmark_data), BATCH_SIZE)):
            batch = benchmark_data[i:i+BATCH_SIZE]
            prompts = [generate_conversation(row) for row in batch]
            chat_prompts = tokenizer.apply_chat_template(
                prompts,
                tokenize=False,
                add_generation_prompt=True
            )
                
            inputs = tokenizer(
                chat_prompts,
                padding=True,
                truncation=True,
                return_tensors="pt",
                max_length=max_seq_length
            ).to(device)
            
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=1024,
                    do_sample=False,
                    temperature=None,
                    top_p=None,
                    top_k=None
                )
            
            decoded = tokenizer.batch_decode(
                outputs[:, inputs.input_ids.shape[1]:], 
                skip_special_tokens=True,
            )
            
            for j, item in enumerate(batch):
                # print(batch[j]['table_data']['Вопрос'])
                # print(f"---------------------------------------------------\n{decoded[j]}\n---------------------------------------------------")
                # print(batch[j]['table_data']['Ответ'])
                # print()
                # print()
                # print()

                item["model_answer"] = decoded[j]

            # assert(0 == 1)


        with open(f"generation{folder_name}/{MODEL_NAME}_{type}.json", "w") as f:
            json.dump(benchmark_data, f, ensure_ascii=False, indent=2)

        torch.cuda.empty_cache()

large


100%|██████████| 58/58 [2:07:40<00:00, 132.07s/it]  


medium


100%|██████████| 58/58 [1:02:28<00:00, 64.63s/it]


small


100%|██████████| 58/58 [35:32<00:00, 36.77s/it]


large


  0%|          | 0/58 [00:40<?, ?it/s]


KeyboardInterrupt: 